# Data Importer
[![Static Badge](https://img.shields.io/badge/Jupyter_Notebook-F37726?style=for-the-badge)](https://jupyter.org/)

<br>

Performs any **adhoc** database operations that need to be done in bulk. 

<br>

## Requirements
- Python (Version 3.6 or up)

<br>
<br>

## Installation
Run the pip install command below:

In [ ]:
%pip install -r requirements.txt

<br>
<br>

## Initialization

Run the codeblock below to initialize all the necessary tools

<br>

> ***❇️ Important*** <br>
>
> You may need to restart the kernel of this notebook if you changed anything in the source code
> 

In [1]:
import sys

sys.path.insert(1, r"src")

import DataImporter as DI

        
########
# MAIN #
########
Secrets = DI.DBSecrets.load()
Database = DI.DBNames.Toy.value
importer = DI.Importer(Secrets, database = Database)


<br>
<br>

## Dataset Format
A *dataset* is a folder that contains many .csv files.<br> 
For simplicity, each .csv file references a particular table. 

<br>

> ***📝 NOTE:*** <br>
>
> The id keys in the .csv files are only for convenience of debugging
> 
> These ids will be regenerated on the database side.


<br>
<br>

## Importing a Dataset

The following codeblock gives some example of importing a dataset

In [2]:
print("===== STARTING TO IMPORT DATA ========")

importer.importData(r"data/Toy Dataset")

print("========== IMPORT COMPLETE ===========")

===== STARTING TO IMPORT DATA ========
Inserting User Data...
Inserting Building Data...
Inserting Room Data...
Inserting Booking Data...
Inserting Cancellation Data...
========== IMPORT COMPLETE ===========


<br>
<br>

## Clearing all Data

The following codeblock gives an example of clearing the data from all the tables

<br>

> ***❗ WARNING:*** <br>
>
> ONLY DO THIS IF YOU ARE ABSOLUTELY SURE OF WHAT YOU ARE DOING
> 

In [ ]:
print("===== STARTING TO DELETE DATA ========")

importer.clearAll()

print("========= DELETION COMPLETE ==========")

===== STARTING TO DELETE DATA ========
Clearing User...
Clearing Building...
Clearing Room...
Clearing Booking...
========= DELETION COMPLETE ==========


<br>
<br>

## Deleting Data from a Particular Table

The following codeblock gives some example of clearing all data from a certain table. <br>
Most likely, you would need to change the below to fit your needs of clearing data.

<br>

> ***❗ WARNING:*** <br>
>
> ONLY DO THIS IF YOU ARE ABSOLUTELY SURE OF WHAT YOU ARE DOING
> 

In [ ]:
print("===== STARTING TO DELETE DATA ========")

importer.clearTable('User')

print("========= DELETION COMPLETE ==========")

===== STARTING TO DELETE DATA ========
========= DELETION COMPLETE ==========


<br>
<br>

## Execute Custom SQL

The following codeblock gives some example to execute some custom sql command

In [6]:
import psycopg2
import psycopg2.sql


selectNameSQL = psycopg2.sql.SQL("SELECT {nameCol} FROM {userTable} LIMIT 1").format(
    nameCol = psycopg2.sql.Identifier("name"),
    userTable = psycopg2.sql.Identifier(DI.TableNames.User.value)
)

conn, cursor, err = importer.executeSQL(selectNameSQL, closeConn = False)

if (err is None):
    print(cursor.fetchone())
    conn.close()
else:
    conn.close()
    raise err

('Alice Margatroid',)
